# Sparkling Water Pipeline Productionalization

## Background

Sparkling Water provides access to H2O algorithms and publishes an API to integrate them as part of regular Spark pipelines. This feature allows for seamless training and deployment of H2O algorithms in the Spark environment. Furthermore, thanks to MOJO (java binary) representations of trained H2O models, production pipelines do not require access to H2O runtime. This enables a wide variety of deployment scenarios. Similarly, Sparkling Water can be used for deploying MOJOs from Driverless AI models.

Moreover, by supporting Python and Scala environments, we enable a simple transfer of modeling results between data scientists ("Python land") and production ("JVM land").


## Goal

The goals of this hands-on are two-fold:
  - Show integration of H2O models into Spark pipelines using PySpark and PySparkling,
  - Demonstrate deployment of the trained pipeline in the context of JVM and Spark streaming.
  
Our modeling goal is to predict sentiment of Amazon food reviews. For this purpose, we use a pre-processed dataset from [SNAP repository](https://snap.stanford.edu/data/web-FineFoods.html). The dataset contains multiple columns but for simplicity, we will use only `date`, `summary` and overall `score`. The score helps us to approximate sentiment.

![Scenario](./img/scenario.png)

## Environment preparation

First, let's verify that `SparkSession` is available in the notebook environment. We do not need to explicitly create a `SparkSession` as it is automatically created for us
during startup of the Jupyter notebook. This works because Jupyter is configured with a Spark kernel.


In [1]:
spark

### Prepare `H2OContext`

We will start `H2OContext` in the so-called _internal backend_ mode. The means H2O is sharing the JVM with Spark, as shown below.

![InternalBackend](./img/internal_backend.png)

The following call initializes H2O on each Spark executor in the Spark cluster.

In [2]:
from pysparkling import *
hc = H2OContext.getOrCreate(spark)

Connecting to H2O server at http://172.17.0.2:54321/h2o ... successful.


H2O cluster uptime:,06 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.5
H2O cluster version age:,2 months and 23 days
H2O cluster name:,sparkling-water-h2o_local-1568217527506
H2O cluster total nodes:,1
H2O cluster free memory:,7.076 Gb
H2O cluster total cores:,16
H2O cluster allowed cores:,16
H2O cluster status:,"accepting new members, healthy"



Sparkling Water Context:
 * Sparkling Water Version: 2.4.13
 * H2O name: sparkling-water-h2o_local-1568217527506
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (driver,3d915dbc845f,54321)
  ------------------------

  Open H2O Flow in browser: http://172.17.0.2:54321/h2o (CMD + click in Mac OSX)

    


>Note: the reported IP is the private IP of the docker container where the notebook is running. To open H2O Flow in your own browser, copy your browser URL and replace the port with 54321. In Aquarium you would replace `/jupyter` with `/h2o`, since Aquarium uses reverse portmapping to get to the standard ports.
>
>For example, my Jupyter notebook's URL is `http://52.202.98.125/jupyter`. After opening a new browser tab or window, copy the address and replace port `/jupyter` with `/h2o`:
>`http://52.202.98.125/h2o`.

## Spark Steps

We are going to use Spark for all the initial steps, from data ingestion to data cleaning and feature engineering.

>Alternately, one could load the data into H2O 
>
>```
import h2o
reviews_h2o = h2o.upload_file("AmazonReviews_Train.csv", "reviews.hex")
>```
>
>use H2O Flow to investigate the data, and then convert the H2O frame to a Spark frame
>
>```
reviews_spark = hc.as_spark_frame(reviews_h2o)
>```

In [3]:
reviews_spark = spark.read.load("/home/h2o/data/amazon_reviews/AmazonReviews_Train.csv",
                                format="csv", sep=",", inferSchema="true", header="true")

#### Trick #1: Save the original Spark schema

At this point, we will save the input data schema to be used later in the deployed Spark streaming application.

In [4]:
reviews_spark.printSchema()

with open('schema.json','w') as f:
    f.write(str(reviews_spark.schema.json()))

root
 |-- Id: integer (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- ProfileName: string (nullable = true)
 |-- HelpfulnessNumerator: string (nullable = true)
 |-- HelpfulnessDenominator: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)



## Now let's define all the stages for the pipeline

The Spark pipelines are composed of various transformers. In our example, we combine a few Spark transformers to clean up textual data and transform it into numerical format. The pipeline is finalized by training an H2O XGBoost binomial model.

> Note: The pipeline stages are not executed right away, they are executed during each fit and transform call.

### Define transformer to drop unnecessary columns
The Spark `SQLTransformer` allows for using SQL to munge data.

As part of this transformer, we convert timestamp to a human readable date string.

For this example, we are selecting just the `Score`, `Time` and `Summary` columns. The goal of this analysis is to predict sentiment, i.e., whether the review is positive or negative. The review can be influenced by several aspects. The `Summary` is of course the mostly important information, but `Time` can influence the model as well. For example, people may tend to give higher reviews on Friday evenings because there's a weekend in front of them. :)

In [5]:
from pyspark.ml.feature import SQLTransformer
colSelect = SQLTransformer(
    statement="SELECT Score, from_unixtime(Time) as Time, Summary FROM __THIS__")

#### Trick #2: Explore intermediate results
To explore intermediate results, we can invoke the defined transformer directly. Note that this will cause Spark to execute the transformer as well as all unevaluated upstream code. 

In [6]:
selected = colSelect.transform(reviews_spark)
selected.show()

+-----+-------------------+--------------------+
|Score|               Time|             Summary|
+-----+-------------------+--------------------+
|    5|2011-04-27 00:00:00|Good Quality Dog ...|
|    1|2012-09-07 00:00:00|   Not as Advertised|
|    4|2008-08-18 00:00:00|"""Delight"" says...|
|    2|2011-06-13 00:00:00|      Cough Medicine|
|    5|2012-10-21 00:00:00|         Great taffy|
|    4|2012-07-12 00:00:00|          Nice Taffy|
|    5|2012-06-20 00:00:00|Great!  Just as g...|
|    5|2012-05-03 00:00:00|Wonderful, tasty ...|
|    5|2011-11-23 00:00:00|          Yay Barley|
|    5|2012-10-26 00:00:00|    Healthy Dog Food|
|    5|2005-02-08 00:00:00|The Best Hot Sauc...|
|    5|2010-08-27 00:00:00|"My cats LOVE thi...|
|    1|2012-06-13 00:00:00|My Cats Are Not F...|
|    4|2010-11-05 00:00:00|   fresh and greasy!|
|    5|2010-03-12 00:00:00|Strawberry Twizzl...|
|    5|2009-12-29 00:00:00|Lots of twizzlers...|
|    2|2012-09-20 00:00:00|          poor taste|
|    5|2012-08-16 00

### Define transformer to create multiple time features based on the `Time` column

The `Time` column is stored internally as a timestamp. To be useful in modeling, we need to extract the time information in a format that is understandable by the predictive algorithms we employ. We can use SparkSQL data methods such as `month`, `dayofmonth`, etc. to engineer multiple new features from the timestamp information. 

In [7]:
refineTime = SQLTransformer(
    statement="""
    SELECT  Score,
            Summary, 
            dayofmonth(Time) as Day, 
            month(Time) as Month, 
            year(Time) as Year, 
            weekofyear(Time) as WeekNum, 
            date_format(Time, 'EEE') as Weekday, 
            hour(Time) as HourOfDay, 
            IF(date_format(Time, 'EEE')='Sat' OR date_format(Time, 'EEE')='Sun', 1, 0) as Weekend, 
            CASE 
                WHEN month(TIME)=12 OR month(Time)<=2 THEN 'Winter' 
                WHEN month(TIME)>=3 OR month(Time)<=5 THEN 'Spring' 
                WHEN month(TIME)>=6 AND month(Time)<=9 THEN 'Summer' 
                ELSE 'Fall' END as Season 
    FROM __THIS__""")

Now inspect the updated data

In [8]:
refined = refineTime.transform(selected)
refined.show()

+-----+--------------------+---+-----+----+-------+-------+---------+-------+------+
|Score|             Summary|Day|Month|Year|WeekNum|Weekday|HourOfDay|Weekend|Season|
+-----+--------------------+---+-----+----+-------+-------+---------+-------+------+
|    5|Good Quality Dog ...| 27|    4|2011|     17|    Wed|        0|      0|Spring|
|    1|   Not as Advertised|  7|    9|2012|     36|    Fri|        0|      0|Spring|
|    4|"""Delight"" says...| 18|    8|2008|     34|    Mon|        0|      0|Spring|
|    2|      Cough Medicine| 13|    6|2011|     24|    Mon|        0|      0|Spring|
|    5|         Great taffy| 21|   10|2012|     42|    Sun|        0|      1|Spring|
|    4|          Nice Taffy| 12|    7|2012|     28|    Thu|        0|      0|Spring|
|    5|Great!  Just as g...| 20|    6|2012|     25|    Wed|        0|      0|Spring|
|    5|Wonderful, tasty ...|  3|    5|2012|     18|    Thu|        0|      0|Spring|
|    5|          Yay Barley| 23|   11|2011|     47|    Wed|      

### Remove neutral reviews and classify the Scores

We are not interested in the neutral reviews (reviews with the `Score=3`) as they would not add much information to the model. This is a fairly standard approach in NPS (net promoter score) type analyses, and common in particular in sentiment analysis. 

In [9]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col, udf
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, IDF, CountVectorizer

filterScore = SQLTransformer(
    statement="""
    SELECT  IF(Score<3,'NEGATIVE', 'POSITIVE') as Sentiment, Summary, Day, Month, Year,
            WeekNum, Weekday, HourOfDay, Weekend, Season 
    FROM __THIS__ WHERE Score !=3 """)

 Inspect the data

In [10]:
filtered = filterScore.transform(refined)
filtered.show()

+---------+--------------------+---+-----+----+-------+-------+---------+-------+------+
|Sentiment|             Summary|Day|Month|Year|WeekNum|Weekday|HourOfDay|Weekend|Season|
+---------+--------------------+---+-----+----+-------+-------+---------+-------+------+
| POSITIVE|Good Quality Dog ...| 27|    4|2011|     17|    Wed|        0|      0|Spring|
| NEGATIVE|   Not as Advertised|  7|    9|2012|     36|    Fri|        0|      0|Spring|
| POSITIVE|"""Delight"" says...| 18|    8|2008|     34|    Mon|        0|      0|Spring|
| NEGATIVE|      Cough Medicine| 13|    6|2011|     24|    Mon|        0|      0|Spring|
| POSITIVE|         Great taffy| 21|   10|2012|     42|    Sun|        0|      1|Spring|
| POSITIVE|          Nice Taffy| 12|    7|2012|     28|    Thu|        0|      0|Spring|
| POSITIVE|Great!  Just as g...| 20|    6|2012|     25|    Wed|        0|      0|Spring|
| POSITIVE|Wonderful, tasty ...|  3|    5|2012|     18|    Thu|        0|      0|Spring|
| POSITIVE|          

### Tokenize the message

Here we use Spark's [RegexTokenizer](https://spark.apache.org/docs/2.1.0/ml-features.html#tokenizer) to tokenize the messages.

In [11]:
regexTokenizer = RegexTokenizer(inputCol="Summary",
                                outputCol="tokenized_summary",
                                pattern="[, ]",
                                toLowercase=True)

Inspect the data

In [12]:
tokenized = regexTokenizer.transform(filtered)
tokenized.show()

+---------+--------------------+---+-----+----+-------+-------+---------+-------+------+--------------------+
|Sentiment|             Summary|Day|Month|Year|WeekNum|Weekday|HourOfDay|Weekend|Season|   tokenized_summary|
+---------+--------------------+---+-----+----+-------+-------+---------+-------+------+--------------------+
| POSITIVE|Good Quality Dog ...| 27|    4|2011|     17|    Wed|        0|      0|Spring|[good, quality, d...|
| NEGATIVE|   Not as Advertised|  7|    9|2012|     36|    Fri|        0|      0|Spring|[not, as, adverti...|
| POSITIVE|"""Delight"" says...| 18|    8|2008|     34|    Mon|        0|      0|Spring|["""delight"", sa...|
| NEGATIVE|      Cough Medicine| 13|    6|2011|     24|    Mon|        0|      0|Spring|   [cough, medicine]|
| POSITIVE|         Great taffy| 21|   10|2012|     42|    Sun|        0|      1|Spring|      [great, taffy]|
| POSITIVE|          Nice Taffy| 12|    7|2012|     28|    Thu|        0|      0|Spring|       [nice, taffy]|
| POSITIVE

### Remove unnecessary words

Some words do not bring much information for the resulting model. For this, we use Spark's [StopWordsRemover](https://spark.apache.org/docs/2.1.0/ml-features.html#stopwordsremover) to clean the data.

In [13]:
stopWordsRemover = StopWordsRemover(inputCol=regexTokenizer.getOutputCol(),
                                    outputCol="CleanedSummary",
                                    caseSensitive=False)

Inspect the data

In [14]:
stopWordsRemoved = stopWordsRemover.transform(tokenized)
stopWordsRemoved.select(["Sentiment", "Summary", "CleanedSummary"]).show()

+---------+--------------------+--------------------+
|Sentiment|             Summary|      CleanedSummary|
+---------+--------------------+--------------------+
| POSITIVE|Good Quality Dog ...|[good, quality, d...|
| NEGATIVE|   Not as Advertised|        [advertised]|
| POSITIVE|"""Delight"" says...|["""delight"", sa...|
| NEGATIVE|      Cough Medicine|   [cough, medicine]|
| POSITIVE|         Great taffy|      [great, taffy]|
| POSITIVE|          Nice Taffy|       [nice, taffy]|
| POSITIVE|Great!  Just as g...|[great!, good, ex...|
| POSITIVE|Wonderful, tasty ...|[wonderful, tasty...|
| POSITIVE|          Yay Barley|       [yay, barley]|
| POSITIVE|    Healthy Dog Food|[healthy, dog, food]|
| POSITIVE|The Best Hot Sauc...|[best, hot, sauce...|
| POSITIVE|"My cats LOVE thi...|["my, cats, love,...|
| NEGATIVE|My Cats Are Not F...|[cats, fans, new,...|
| POSITIVE|   fresh and greasy!|    [fresh, greasy!]|
| POSITIVE|Strawberry Twizzl...|[strawberry, twiz...|
| POSITIVE|Lots of twizzlers

### Hash the words

NLP (natural language processing) for predictive modeling is based on the idea that text can be represented as numeric values. These values are then fed into any algorithm the user chooses. One choice of numeric representation uses [CountVectorizer](https://spark.apache.org/docs/2.1.0/ml-features.html#countvectorizer).

`CountVectorizer` is very similar to the [HashingTF](https://spark.apache.org/docs/2.1.0/ml-features.html#tf-idf) function, except that it preserves the mapping from the index back to the word using an internal vocabulary.

For example, if the word `Dog` is stored in the hash at the index `100`, we can get the word back as `countVectorizerModel.vocabulary[100]`.

#### Trick #3: Set minDF parameter to limit number of words

The `minDF` parameter ensures that only words which occur more the `minDF` times in our case are included. This both speeds the process of modeling and ensures that outliers (infrequent words) do not affect our model that much.

In [15]:
countVectorizer = CountVectorizer(inputCol=stopWordsRemover.getOutputCol(),
                                  outputCol="frequencies",
                                  minDF=100)

#### Trick #4: Manually train the count vectorizer so we can see how it behaves before we execute the pipeline


In [16]:
countVecModel = countVectorizer.fit(stopWordsRemoved)

See the vocabulary:

In [17]:
print("Vocabulary size is " + str(len(countVecModel.vocabulary)))
print(countVecModel.vocabulary[:10])

Vocabulary size is 1513
['great', 'good', 'best', 'love', 'coffee', 'tea', 'product', 'taste', 'delicious', 'excellent']


Inspect the data

In [18]:
vectorized = countVecModel.transform(stopWordsRemoved)
vectorized.select(["Sentiment", "CleanedSummary", "frequencies"]).show()

+---------+--------------------+--------------------+
|Sentiment|      CleanedSummary|         frequencies|
+---------+--------------------+--------------------+
| POSITIVE|[good, quality, d...|(1513,[1,10,12,35...|
| NEGATIVE|        [advertised]|  (1513,[617],[1.0])|
| POSITIVE|["""delight"", sa...|  (1513,[418],[1.0])|
| NEGATIVE|   [cough, medicine]|        (1513,[],[])|
| POSITIVE|      [great, taffy]|(1513,[0,1430],[1...|
| POSITIVE|       [nice, taffy]|(1513,[30,1430],[...|
| POSITIVE|[great!, good, ex...|(1513,[1,59,126],...|
| POSITIVE|[wonderful, tasty...|(1513,[15,37,1430...|
| POSITIVE|       [yay, barley]|        (1513,[],[])|
| POSITIVE|[healthy, dog, food]|(1513,[10,12,21],...|
| POSITIVE|[best, hot, sauce...|(1513,[2,44,86,45...|
| POSITIVE|["my, cats, love,...|(1513,[3,12,23,41...|
| NEGATIVE|[cats, fans, new,...|(1513,[12,41,80],...|
| POSITIVE|    [fresh, greasy!]|   (1513,[83],[1.0])|
| POSITIVE|[strawberry, twiz...|(1513,[13,19,666]...|
| POSITIVE|[lots, twizzlers,

### Create an Inverse Document Frequency (IDF) model

Here we use Spark's [tf-idf](https://spark.apache.org/docs/2.1.0/ml-features.html#tf-idf) method to model the importance of a term in a document to the given set of data. Please see the [Spark documentation](https://spark.apache.org/docs/2.1.0/ml-features.html#tf-idf) for more information on TF-IDF.

In [19]:
idf = IDF(inputCol=countVectorizer.getOutputCol(),
          outputCol="tf_idf_frequencies",
          minDocFreq=1)

Manually train the IDF model to see the results before we execute the pipeline,

In [20]:
idfModel = idf.fit(vectorized)

Inspect the data

In [21]:
afterIdf = idfModel.transform(vectorized)
afterIdf.select(["Sentiment", "CleanedSummary", "frequencies", "tf_idf_frequencies"]).show()

+---------+--------------------+--------------------+--------------------+
|Sentiment|      CleanedSummary|         frequencies|  tf_idf_frequencies|
+---------+--------------------+--------------------+--------------------+
| POSITIVE|[good, quality, d...|(1513,[1,10,12,35...|(1513,[1,10,12,35...|
| NEGATIVE|        [advertised]|  (1513,[617],[1.0])|(1513,[617],[7.32...|
| POSITIVE|["""delight"", sa...|  (1513,[418],[1.0])|(1513,[418],[6.91...|
| NEGATIVE|   [cough, medicine]|        (1513,[],[])|        (1513,[],[])|
| POSITIVE|      [great, taffy]|(1513,[0,1430],[1...|(1513,[0,1430],[2...|
| POSITIVE|       [nice, taffy]|(1513,[30,1430],[...|(1513,[30,1430],[...|
| POSITIVE|[great!, good, ex...|(1513,[1,59,126],...|(1513,[1,59,126],...|
| POSITIVE|[wonderful, tasty...|(1513,[15,37,1430...|(1513,[15,37,1430...|
| POSITIVE|       [yay, barley]|        (1513,[],[])|        (1513,[],[])|
| POSITIVE|[healthy, dog, food]|(1513,[10,12,21],...|(1513,[10,12,21],...|
| POSITIVE|[best, hot, sa

### Remove Summary Column

Recall from above that predictive algorithms do not understand string values very well. This is why we transformed the text data of the `Summary` column using TF-IDF. We will keep the numeric representations of `Summary` and drop the original text so that we do not confuse the model.

In [22]:
removeSummary = SQLTransformer(
    statement="""
    SELECT Sentiment, Day, Month, Year, WeekNum, Weekday, HourOfDay, Weekend, Season, tf_idf_frequencies
    FROM __THIS__ """)

Inspect the data

In [23]:
removedSummary = removeSummary.transform(afterIdf)
removedSummary.show()

+---------+---+-----+----+-------+-------+---------+-------+------+--------------------+
|Sentiment|Day|Month|Year|WeekNum|Weekday|HourOfDay|Weekend|Season|  tf_idf_frequencies|
+---------+---+-----+----+-------+-------+---------+-------+------+--------------------+
| POSITIVE| 27|    4|2011|     17|    Wed|        0|      0|Spring|(1513,[1,10,12,35...|
| NEGATIVE|  7|    9|2012|     36|    Fri|        0|      0|Spring|(1513,[617],[7.32...|
| POSITIVE| 18|    8|2008|     34|    Mon|        0|      0|Spring|(1513,[418],[6.91...|
| NEGATIVE| 13|    6|2011|     24|    Mon|        0|      0|Spring|        (1513,[],[])|
| POSITIVE| 21|   10|2012|     42|    Sun|        0|      1|Spring|(1513,[0,1430],[2...|
| POSITIVE| 12|    7|2012|     28|    Thu|        0|      0|Spring|(1513,[30,1430],[...|
| POSITIVE| 20|    6|2012|     25|    Wed|        0|      0|Spring|(1513,[1,59,126],...|
| POSITIVE|  3|    5|2012|     18|    Thu|        0|      0|Spring|(1513,[15,37,1430...|
| POSITIVE| 23|   11|

------------
# Now use H2O

### Create an XGBoost model using H2O

Up to this point, all of our data wrangling and feature engineering efforts have used Spark methods exclusively. Now we turn to H2O to train an H2O XGBoost model on the `Sentiment` column (using default settings). Note that there are many more steps involved with tuning an XGBoost model which we omit here. The full documentation for XGBoost is available at [H2O Documentation](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/xgboost.html)

In [24]:
import h2o
from pysparkling.ml import ColumnPruner, H2OXGBoost

xgboost = H2OXGBoost(splitRatio=0.8,
             featuresCols=[idf.getOutputCol()],
             labelCol="Sentiment")

###  Create the pipeline by defining all the stages

Now we have all the pieces ready and can define the final pipeline.

In [25]:
pipeline = Pipeline(stages=[colSelect,
                            refineTime,
                            filterScore,
                            regexTokenizer,
                            stopWordsRemover,
                            countVectorizer,
                            idf,
                            removeSummary,
                            xgboost])

## Train the pipeline model

The `fit` call calls each trasformer and estimator in the pipeline and creates so called the `PipelineModel`. The model is trained from the cleaned data from previous transformers and the final model is ready to accept the raw data to make predictions

In [26]:
model = pipeline.fit(reviews_spark)

### Try predictions

First, let's load the data that we can use for prediction

In [27]:
reviews_spark_pred = spark.read.load("/home/h2o/data/amazon_reviews/AmazonReviews_Predictions.csv",
                                     format="csv", sep=",", inferSchema="true", header="true")

Now run the predictions:

In [28]:
model.transform(reviews_spark_pred).show()

+---------+---+-----+----+-------+-------+---------+-------+------+--------------------+--------------------+
|Sentiment|Day|Month|Year|WeekNum|Weekday|HourOfDay|Weekend|Season|  tf_idf_frequencies|   prediction_output|
+---------+---+-----+----+-------+-------+---------+-------+------+--------------------+--------------------+
| POSITIVE|  8|    6|2012|     23|    Fri|        0|      0|Spring|(1513,[48,645,130...|[0.21973294019699...|
| POSITIVE| 15|   12|2011|     50|    Thu|        0|      0|Winter|        (1513,[],[])|[0.21973294019699...|
| POSITIVE| 14|    9|2011|     37|    Wed|        0|      0|Spring|(1513,[258,307],[...|[0.21973294019699...|
| POSITIVE| 20|   10|2011|     42|    Thu|        0|      0|Spring|(1513,[26,445],[4...|[0.21973294019699...|
| POSITIVE|  9|    9|2012|     36|    Sun|        0|      1|Spring|(1513,[36,1409],[...|[0.21973294019699...|
| POSITIVE|  8|    2|2012|      6|    Wed|        0|      0|Winter|        (1513,[],[])|[0.21973294019699...|
| NEGATIVE

## Save the pipeline model

Later we can use the pipeline model in Scala to demonstrate the deployment of the pipeline in the JVM world.

In [29]:
model.write().overwrite().save("reviews_pipeline.model")

#### Trick #5: Check variable importances

We can inspect the model in H2O Flow and see the variable importances. However, we do not have information about the words, just the indices. We can ask the `CountVectorizer` what word is on the specific index to see what words affect our model the most.

In [30]:
model.stages[5].vocabulary[0]

'great'

## Let's Deploy the Application

Up to this point, we have defined the PySpark pipeline. We will now demonstrate its deployment using the PySpark Streaming application in python, where the pipeline defined above will receive raw streaming data and run predictions on them in real time.

The steps will be:

 1. Load the schema from the schema file.
 1. Load the pipeline from the pipeline file.
 1. Create an input data stream and pass it the schema. The input data stream will point to a directory where a new csv files will be coming from different streaming sources.
 1. Create and output the data stream. For the purposes of this tutorial, we store the data into memory and also into a SparkSQL table.
 1. We can inspect the predictions in "real time" by regularly displaying the content of the desired table.

### 1. Check again that we have spark available

In [31]:
spark

### 2. Load the exported schema of our input data

In [32]:
from pyspark.sql.types import StructType
import json

schema = StructType.fromJson(json.load(open("schema.json", 'r')))
print(schema)

StructType(List(StructField(Id,IntegerType,true),StructField(ProductId,StringType,true),StructField(UserId,StringType,true),StructField(ProfileName,StringType,true),StructField(HelpfulnessNumerator,StringType,true),StructField(HelpfulnessDenominator,StringType,true),StructField(Score,StringType,true),StructField(Time,StringType,true),StructField(Summary,StringType,true),StructField(Text,StringType,true)))


### 3. Load the exported pipeline model

In [33]:
from pyspark.ml import PipelineModel
pipeline_model = PipelineModel.load("reviews_pipeline.model/")

### 4. Start Streaming

In [34]:
from subprocess import Popen
Popen(["./start_streaming.sh"])

In [35]:
!ls output

0.csv  1.csv


### 5. Prepare the input data stream

In [36]:
input_data_stream = spark.readStream.schema(schema).csv("output")

### 6. Prepare the output data stream

In [37]:
output_data_stream = pipeline_model.transform(input_data_stream)

### 7. Start processing the input data

In [38]:
output_data_stream.writeStream.format("memory").queryName("predictions").start()

### 8. List the output

In [39]:
import time
# limit to 10 for hands-on
# could replace with 
# while(True):

for x in range(10):
    spark.sql("select * from predictions").show()
    time.sleep(3)

+---------+---+-----+----+-------+-------+---------+-------+------+--------------------+--------------------+
|Sentiment|Day|Month|Year|WeekNum|Weekday|HourOfDay|Weekend|Season|  tf_idf_frequencies|   prediction_output|
+---------+---+-----+----+-------+-------+---------+-------+------+--------------------+--------------------+
| POSITIVE| 18|    8|2008|     34|    Mon|        0|      0|Spring|(1513,[419],[6.91...|[0.21973294019699...|
| POSITIVE| 12|    7|2012|     28|    Thu|        0|      0|Spring|(1513,[30,1426],[...|[0.21973294019699...|
| POSITIVE| 20|    6|2012|     25|    Wed|        0|      0|Spring|(1513,[1,59,126],...|[0.21973294019699...|
| POSITIVE| 27|    4|2011|     17|    Wed|        0|      0|Spring|(1513,[1,10,12,35...|[0.21973294019699...|
| NEGATIVE| 13|    6|2011|     24|    Mon|        0|      0|Spring|        (1513,[],[])|[0.21973294019699...|
| NEGATIVE|  7|    9|2012|     36|    Fri|        0|      0|Spring|(1513,[618],[7.32...|[0.21973294019699...|
| POSITIVE

### 9. Shut down the cluster

In [40]:
h2o.cluster().shutdown()
spark.stop()